<a href="https://colab.research.google.com/github/Rad-Monk/TF-Notebooks/blob/main/dfd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d ahmadawad732/uadfv-dataset-new

In [ ]:
!unzip uadfv-dataset-new.zip

In [ ]:
!pip install albumentations
!pip install cv2

In [ ]:
import os

import keras
from imutils import paths

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import imageio
import cv2
from IPython.display import Image
from tensorflow.keras.applications import EfficientNetB4
from albumentations import Compose, ImageCompression, GaussNoise, GaussianBlur, HorizontalFlip, PadIfNeeded, OneOf, RandomBrightnessContrast, FancyPCA, HueSaturationValue, ToGray, ShiftScaleRotate, Resize
import albumentations.augmentations.functional as F
import cv2

In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 20

MAX_SEQ_LENGTH = 250
NUM_FEATURES = 1792 #for efficientnetb4
# NUM_FEATURES = 2048 #for inceptionv3

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

train_data = []
val_data = []
test_data = []

### DATASET 1 (FaceForensics + Celeb)
import os
for dirname, _, filenames in os.walk('/content/fake_videos/'):
    for filename in filenames:
        if 'validation' in dirname:
            val_data.append(os.path.join(dirname, filename))
        elif 'train' in dirname:
            train_data.append(os.path.join(dirname, filename))
        else:
            test_data.append(os.path.join(dirname, filename))

In [ ]:
train_data[:10]

In [ ]:
train_df = pd.DataFrame(train_data)
val_df = pd.DataFrame(val_data)
test_df = pd.DataFrame(test_data)

In [ ]:
train_df

In [ ]:
def label_file(row):
    if 'real' in row:
        return "real"
    elif 'fake' in row:
        return "fake"
    else:
        return 'undefined'

In [ ]:
train_df['label'] = train_df[0].apply(label_file)
val_df['label'] = val_df[0].apply(label_file)
test_df['label'] = test_df[0].apply(label_file)

In [ ]:
train_df

In [ ]:
print(train_df['label'].value_counts())
print(val_df['label'].value_counts())
print(test_df['label'].value_counts())

In [ ]:
def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

In [ ]:
from tensorflow import keras
from tensorflow.keras.applications import EfficientNetB4

IMG_SIZE = 224  # Adjust as necessary for EfficientNetB4

def build_feature_extractor():
    # Create the base EfficientNetB4 model with pre-trained weights from ImageNet
    feature_extractor = EfficientNetB4(
        weights="imagenet",
        include_top=False,  # Exclude the classification head
        pooling="avg",  # Use global average pooling to get a feature vector
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )

    # Define the model's input
    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))

    # The EfficientNet model will automatically handle the input preprocessing
    outputs = feature_extractor(inputs)

    # Create a Keras Model with the specified input and output
    return keras.Model(inputs, outputs, name="feature_extractor")

# Instantiate the feature extractor
feature_extractor = build_feature_extractor()



# def build_feature_extractor():
#     feature_extractor = keras.applications.InceptionV3(
#         weights="imagenet",
#         include_top=False,
#         pooling="avg",
#         input_shape=(IMG_SIZE, IMG_SIZE, 3),
#     )
#     preprocess_input = keras.applications.inception_v3.preprocess_input

#     inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
#     preprocessed = preprocess_input(inputs)

#     outputs = feature_extractor(preprocessed)
#     return keras.Model(inputs, outputs, name="feature_extractor")


# feature_extractor = build_feature_extractor()

In [ ]:
train_df

In [ ]:
test_df

In [ ]:
train_df.rename(columns={0:'filepath'}, inplace=True)
test_df.rename(columns={0:'filepath'}, inplace=True)
val_df.rename(columns={0:'filepath'}, inplace=True)

In [ ]:
label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train_df["label"])
)
print(label_processor.get_vocabulary())

In [ ]:
train_df

In [ ]:
test_df

In [ ]:

# def IsotropicResize(image, size=380):
#     # Compute the scaling factor to resize the image while preserving the aspect ratio
#     h, w = image.shape[:2]
#     scale = size / max(h, w)
#     resized_image = cv2.resize(image, (int(w * scale), int(h * scale)), interpolation=cv2.INTER_LINEAR)

#     # Pad the image if necessary to ensure it's square
#     if h > w:
#         pad_w = (size - resized_image.shape[1]) // 2
#         pad_h = 0
#     else:
#         pad_h = (size - resized_image.shape[0]) // 2
#         pad_w = 0

#     padded_image = cv2.copyMakeBorder(resized_image, pad_h, pad_h, pad_w, pad_w, cv2.BORDER_CONSTANT)
#     return padded_image

# # Function to apply transformations, including the custom isotropic resize
# def create_train_transforms(size=380):
#     return Compose([
#         ImageCompression(quality_lower=60, quality_upper=100, p=0.5),
#         GaussNoise(p=0.1),
#         GaussianBlur(blur_limit=3, p=0.05),
#         HorizontalFlip(),
#         Resize(height=size, width=size, always_apply=True),  # Resize to ensure the final output is consistent
#         OneOf([RandomBrightnessContrast(), FancyPCA(), HueSaturationValue()], p=0.7),
#         ToGray(p=0.2),
#         ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=10, border_mode=cv2.BORDER_CONSTANT, p=0.5),
#     ], additional_targets={'image': 'image'})

# def apply_isotropic_resize(image, size=380):
#     return IsotropicResize(image, size=size)

In [ ]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["filepath"].values.tolist()
    labels = df["label"].values
    labels = keras.ops.convert_to_numpy(label_processor(labels[..., None]))

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(
            shape=(
                1,
                MAX_SEQ_LENGTH,
            ),
            dtype="bool",
        )
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :], verbose=0,
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df, "train")
test_data, test_labels = prepare_all_videos(test_df, "test")
val_data, val_labels = prepare_all_videos(val_df, "val")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")

Frame features in train set: (82, 250, 1792)
Frame masks in train set: (82, 250)


In [ ]:
# import tensorflow as tf
# import numpy as np

# class VideoDataGenerator(tf.keras.utils.Sequence):
#     def __init__(self, df, root_dir, batch_size, max_seq_length, num_features, label_processor, feature_extractor, shuffle=True, **kwargs):
#         super().__init__(**kwargs)
#         self.df = df
#         self.root_dir = root_dir
#         self.batch_size = batch_size
#         self.max_seq_length = max_seq_length
#         self.num_features = num_features
#         self.label_processor = label_processor
#         self.feature_extractor = feature_extractor
#         self.shuffle = shuffle
#         self.indices = np.arange(len(self.df))
#         self.on_epoch_end()

#     def __len__(self):
#         # Number of batches per epoch
#         return len(self.df) // self.batch_size

#     def __getitem__(self, index):
#         # Generate one batch of data
#         batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
#         batch_df = self.df.iloc[batch_indices]
#         return self.__data_generation(batch_df)

#     def on_epoch_end(self):
#         # Shuffle indices after each epoch if shuffle is set to True
#         if self.shuffle:
#             np.random.shuffle(self.indices)

#     def __data_generation(self, batch_df):
#         # Initialize arrays to hold batch data
#         batch_size = len(batch_df)
#         video_paths = batch_df["filepath"].values.tolist()
#         labels = batch_df["label"].values
#         labels = keras.ops.convert_to_numpy(self.label_processor(labels[..., None]))

#         frame_masks = np.zeros(shape=(batch_size, self.max_seq_length), dtype="bool")
#         frame_features = np.zeros(
#             shape=(batch_size, self.max_seq_length, self.num_features), dtype="float32"
#         )

#         for idx, path in enumerate(video_paths):
#             frames = load_video(os.path.join(self.root_dir, path))
#             frames = frames[None, ...]

#             temp_frame_mask = np.zeros(
#                 shape=(
#                     1,
#                     self.max_seq_length,
#                 ),
#                 dtype="bool",
#             )
#             temp_frame_features = np.zeros(
#                 shape=(1, self.max_seq_length, self.num_features), dtype="float32"
#             )

#             video_length = frames.shape[1]
#             length = min(self.max_seq_length, video_length)
#             for j in range(length):
#                 temp_frame_features[0, j, :] = self.feature_extractor.predict(
#                     frames[:, j, :], verbose=0,
#                 )
#             temp_frame_mask[0, :length] = 1  # 1 = not masked, 0 = masked

#             frame_features[idx,] = temp_frame_features.squeeze()
#             frame_masks[idx,] = temp_frame_mask.squeeze()

#         return (frame_features, frame_masks), labels


In [ ]:
# train_generator = VideoDataGenerator(
#     df=train_df,
#     root_dir="train",
#     batch_size=32,  # Adjust batch size as needed
#     max_seq_length=MAX_SEQ_LENGTH,
#     num_features=NUM_FEATURES,
#     label_processor=label_processor,
#     feature_extractor=feature_extractor,
#     shuffle=True
# )

# val_generator = VideoDataGenerator(
#     df=val_df,
#     root_dir="val",
#     batch_size=32,
#     max_seq_length=MAX_SEQ_LENGTH,
#     num_features=NUM_FEATURES,
#     label_processor=label_processor,
#     feature_extractor=feature_extractor,
#     shuffle=False
# )

# test_generator = VideoDataGenerator(
#     df=test_df,
#     root_dir="test",
#     batch_size=32,
#     max_seq_length=MAX_SEQ_LENGTH,
#     num_features=NUM_FEATURES,
#     label_processor=label_processor,
#     feature_extractor=feature_extractor,
#     shuffle=False
# )


In [ ]:
print(train_data[0])
# test_data[0]
# val_data[0]

[[[-0.22048397 -0.22533458  0.330482   ... -0.24377686 -0.16279313
   -0.22083683]
  [-0.22043574 -0.22536124  0.33084404 ... -0.24370903 -0.16319561
   -0.2207426 ]
  [-0.22526318 -0.21486829  0.34552708 ... -0.23594657 -0.16810413
   -0.22859141]
  ...
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]]

 [[-0.20833303 -0.14511505  0.5069105  ... -0.17886426 -0.21980815
   -0.19552281]
  [-0.20816542 -0.14492089  0.5060624  ... -0.17869759 -0.21967046
   -0.19546711]
  [-0.21387091 -0.15112539  0.46598092 ... -0.18394992 -0.22213219
   -0.2017795 ]
  ...
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]]

 [[-0.23300855 -0.18729949  0.407202

In [ ]:
train_data[1]
# test_data[1]
# val_data[1]

array([[ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       ...,
       [ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False]])

In [ ]:
# train_labels
# test_labels
# val_labels

In [ ]:
# Utility for our sequence model.
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES), name = "fname_features")
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool", name = "masks")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    # Modify the GRU layers
    x = keras.layers.GRU(64, return_sequences=True)(frame_features_input, mask=mask_input)
    x = keras.layers.GRU(32)(x)
    x = keras.layers.Dropout(0.5)(x)
    x = keras.layers.Dense(32, activation="relu", kernel_regularizer=keras.regularizers.l2(0.001))(x)
    output = keras.layers.Dense(1, activation="sigmoid")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    optimizer = keras.optimizers.Adam(learning_rate=1e-5)

    rnn_model.compile(
        loss="BinaryCrossentropy", optimizer=optimizer, metrics=["accuracy"]
    )



    # Implement gradient clipping
    optimizer = keras.optimizers.Adam(learning_rate=0.0001, clipnorm=1.0)

    return rnn_model


# Utility for running experiments.
def run_experiment():
    filepath = "/tmp/video_classifier/ckpt.weights.h5"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    lr_scheduler = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1
    )


    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_data=(val_data[0], val_data[1], val_labels),
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        callbacks=[checkpoint, lr_scheduler],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    print(accuracy)

    return history, seq_model


_, sequence_model = run_experiment()

Epoch 1/20
1/2 ━━━━━━━━━━━━━━━━━━━━ 2s 3s/step - accuracy: 0.5000 - loss: 0.7247

ValueError: Layer "functional_16" expects 2 input(s), but it received 1 input tensors. Inputs received: [<tf.Tensor 'IteratorGetNext:0' shape=(None, 250, 1792) dtype=float32>]